In [1]:
!pip install wandb

     |████████████████████████████████| 1.4MB 2.8MB/s 
     |████████████████████████████████| 112kB 47.3MB/s 
     |████████████████████████████████| 460kB 47.5MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=2dec55b8b47766d0fd875cd4aec6236c4be6c0d31b1f20da99aec685754426c7
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=f3f0e9b0ea4e4d458f8318fadd3c88db3a3560fa5e8a9fad2de18f2455f7cd86
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=9f

In [2]:
!wandb login

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: dff003aa03e7d25df35a840b6f0660ae9675efb4
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [3]:
#GPU INFO
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue May 12 13:19:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar')

In [6]:
# Pytorch libraries
import torch
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

# Internal files
import config
import dataloader
import models
# from baseline import get_model, save_checkpoint

import math
import os
import sys
import time
from collections import OrderedDict
import random

import wandb



noise file noisy_label_kv40.0.txt generated with noise: 0.4



In [0]:
def get_model():

    # Get model from config
    if config.model == "resnet18":
        model = models.resnet18(pretrained=config.pretrained)
    elif config.model == "resnet34":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnet50':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "resnet101":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "resnet152":
        model = models.resnet152(pretrained=config.pretrained)
    elif config.model == "resnext50_32x4d":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnext101_32x8d':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "wide_resnet50_2":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "wide_resnet101_2":
        model = models.resnet152(pretrained=config.pretrained)
    else:
        raise ValueError('%s not supported'.format(config.model))

    # Initialize fc layer
    (in_features, out_features) = model.fc.in_features, model.fc.out_features
    model.fc = torch.nn.Linear(in_features, out_features)
    return model



def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if config.use_wandb == True:
        wandb.save(filename)

In [0]:
def scheduler(epoch: int):
    global lr
    lr = config.lr
    if epoch > config.start_epoch:
        lr = lr / 10.0
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Training
def train(epoch):
    global init
    net.train()
    tch_net.train()
    train_loss = 0
    correct = 0
    total = 0
    scheduler(epoch)


    # ramp up meta-learning rate and EMA decay
    if epoch <= config.param_epoch:
        u = epoch/config.param_epoch
        meta_lr = config.meta_lr*math.exp(-5*(1-u)**2)
        lamb = 0.5*math.exp(-5*(1-u)**2)
    else:
        meta_lr = config.meta_lr
        config.eps = 0.999

    for step, (inputs, targets) in enumerate(train_loader):
        init_time = time.time()
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)

        class_loss = criterion(outputs, targets)
        class_loss.backward(retain_graph=True)


        if step > config.start_iter or epoch > 1:
        #if step > 0 or epoch > 0:

            # if step > config.mid_iter or epoch > 1:
            #     # config.eps = 0.999
            #     alpha = config.alpha
            # else:
            #     u = (step - config.start_iter)/(config.mid_iter - config.start_iter)
            #     alpha = config.alpha*math.exp(-5*(1-u)**2)
            alpha = config.alpha

            if init:
                init = False
                for param, param_tch in zip(net.parameters(), tch_net.parameters()):
                    param_tch.data.copy_(param.data)
            else:
                for param, param_tch in zip(net.parameters(), tch_net.parameters()):
                    param_tch.data.mul_(config.eps).add_((1-config.eps), param.data)

            _, feats = pretrain_net(inputs, get_feat=True)
            tch_outputs = tch_net(inputs, get_feat=False)
            p_tch = F.softmax(tch_outputs, dim=1)
            p_tch.detach_()

            if use_mentor == True:
                mnt_outputs = mentor_net(inputs, get_feat=False)
                p_mnt = F.softmax(tch_outputs, dim=1)
                p_mnt.detach_()

            for i in range(config.num_fast):
                targets_fast = targets.clone()
                randidx = torch.randperm(targets.size(0))
                for n in range(int(targets.size(0)*config.perturb_ratio)):
                    num_neighbor = 10
                    idx = randidx[n]
                    feat = feats[idx]
                    feat.view(1, feat.size(0))
                    feat.data = feat.data.expand(targets.size(0), feat.size(0))
                    dist = torch.sum((feat-feats)**2, dim=1)
                    _, neighbor = torch.topk(dist.data, num_neighbor+1, largest=False)
                    targets_fast[idx] = targets[neighbor[random.randint(1, num_neighbor)]]

                fast_loss = criterion(outputs, targets_fast)

                grads = torch.autograd.grad(fast_loss, net.parameters(),
                                            create_graph=False,
                                            retain_graph=True,
                                            only_inputs=True)

                fast_weights = OrderedDict(
                    (name, param - meta_lr*grad)
                    for ((name, param), grad) in zip(net.named_parameters(), grads))

                fast_out = net.forward(inputs,fast_weights)

                logp_fast = F.log_softmax(fast_out,dim=1)

                #afegir canvis per iterative aquí
                if use_mentor == False:
                    consistent_loss = consistent_criterion(logp_fast, p_tch)
                else:
                    consistent_loss = consistent_criterion(logp_fast, p_tch*lamb + p_mnt*(1-lamb))

                consistent_loss = consistent_loss*alpha/config.num_fast
                consistent_loss.backward()

        optimizer.step()

        # train_loss += class_loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        # Grab training results
        sys.stdout.write('\r')
        sys.stdout.write('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%, time: %.3f'
              %(epoch, config.num_epochs, step+1, (len(train_loader.dataset)//config.batch_size)+1, class_loss.data.item(), 100.*correct/total,time.time() - init_time))
        sys.stdout.flush()



def valid(epoch, network):
    global best_acc
    network.eval()
    # val_loss = 0
    correct = 0
    total = 0
    for step, (inputs, targets) in enumerate(valid_loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = network(inputs)
            loss = criterion(outputs, targets)

        # valid_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        # Grab validation results
        valid_acc = 100. * correct / total
      # valid_results = ("| Epoch: {}/{}, val_loss: {:.3f}, val_acc: {:.3f}, "
      #                 "lr: {:.6f}".format(epoch,
      #                                     config.num_epochs,
      #                                     loss.data.item(),
      #                                     valid_acc,
      #                                     lr))
        # Grab validation results
        valid_results = ("| Epoch: {}/{}, val_loss: {:.3f}, val_acc: {:.3f}, ""lr: {:.6f}".format(epoch,config.num_epochs,loss.data.item(),valid_acc,lr))
        record.write(valid_results + '\n')
        record.flush()



    # Save checkpoint when best model
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('| Saving Best Model ...', end="\r")
        save_point = config.drive_dir + '/checkpoint/' + config.id + '.pth.tar'
        save_checkpoint({
            'state_dict': network.state_dict(),
            'best_acc': best_acc,
        }, save_point)

    wandb.log({'epoch': epoch, 'accy_val' : best_acc })

    return valid_results


def test():
    test_net.eval()
    # test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(valid_loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = test_net(inputs)
            loss = criterion(outputs, targets)

        # test_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Grab validation results
    test_acc = 100. * correct/total
    test_results = "| test_loss: {:.3f}, test_acc: {:.3f}".format(
        loss.data.item(), test_acc)
    record.write(test_results)
    record.flush()

    print(test_results)


In [0]:
def save_weights(epoch):
        print('| Saving Weights student ...', end="\r")
        save_point = config.drive_dir + '/checkpoint/' + config.id + '_student_' + str(epoch) + '.pth.tar'
        save_checkpoint({'state_dict': net.state_dict(), }, save_point)

        print('| Saving Weights teacher ...', end="\r")
        save_point = config.drive_dir + '/checkpoint/' + config.id + '_teacher_' + str(epoch) + '.pth.tar'
        save_checkpoint({'state_dict': tch_net.state_dict(), }, save_point)

In [10]:
# Checkpoint dir.
record = open(config.drive_dir + '/checkpoint/' + config.checkpoint + '_test.txt', 'w')
record.write('noise_rate=%s\n' % config.noise_rate)
record.flush()

# Get the original_dataset
loader = dataloader.KeyDataLoader()
train_loader, valid_loader, test_loader = loader.run()

# Hyper Parameter settings
random.seed(config.seed)
# torch.cuda.set_device(config.gpuid)
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)
use_cuda = torch.cuda.is_available()

# Networks setup
print('\nModel setup')
print('| Building network: {}'.format(config.model))
net = get_model()
tch_net = get_model()
pretrain_net = get_model()
test_net = get_model()

print('| load pretrained net. from checkpoint...')
checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + '.pth.tar')
pretrain_net.load_state_dict(checkpoint['state_dict'])

if use_cuda:
    net.cuda()
    tch_net.cuda()
    pretrain_net.cuda()
    test_net.cuda()
    cudnn.benchmark = True
pretrain_net.eval()

for param in tch_net.parameters():
    param.requires_grad = False
for param in pretrain_net.parameters():
    param.requires_grad = False

# Instantiate a loss function.
criterion = torch.nn.CrossEntropyLoss()
consistent_criterion = torch.nn.KLDivLoss()

# Instantiate an optimizer to train the model
optimizer = torch.optim.SGD(
    net.parameters(), lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay)

print('\nTraining model')
print('| Training Epochs = ' + str(config.num_epochs))
print('| Initial Learning Rate = ' + str(config.lr))
print('| Optimizer = ' + str(config.optimizer_type))



Model setup
| Building network: resnet34


/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:122: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight, mode='fan_out')
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:124: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.weight, 1)
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:125: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias, 0)


| load pretrained net. from checkpoint...

Training model
| Training Epochs = 120
| Initial Learning Rate = 0.2
| Optimizer = SGD


In [0]:
init = True
best_acc = 0
use_mentor = False
for epoch in range(1, 1 + config.num_epochs):
    train(epoch)
    # Student validation
    std_results = valid(epoch, net)
    record.write(std_results + '\n')
    print(std_results)
    # Teacher validation
    tch_results = valid(epoch, tch_net)
    record.write(tch_results + '\n')
    record.flush()
    print(tch_results)

    save_weights(epoch)


| Epoch: 1/120, val_loss: 2.826, val_acc: 17.920, lr: 0.200000
| Epoch: 1/120, val_loss: 0.003, val_acc: 10.000, lr: 0.200000


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2247: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


| Epoch [  2/120] Iter[  1/352]		Loss: 2.0485 Acc@1: 14.062%, time: 2.379

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


| Epoch: 2/120, val_loss: 2.905, val_acc: 21.800, lr: 0.200000
| Epoch: 2/120, val_loss: 2.723, val_acc: 24.680, lr: 0.200000
| Epoch: 3/120, val_loss: 2.288, val_acc: 29.020, lr: 0.200000
| Epoch: 3/120, val_loss: 2.187, val_acc: 30.800, lr: 0.200000
| Epoch: 4/120, val_loss: 2.336, val_acc: 31.380, lr: 0.200000
| Epoch: 4/120, val_loss: 1.954, val_acc: 36.260, lr: 0.200000
| Epoch [  5/120] Iter[352/352]		Loss: 1.6336 Acc@1: 38.927%, time: 1.175| Epoch: 5/120, val_loss: 2.090, val_acc: 35.380, lr: 0.200000
| Epoch: 5/120, val_loss: 1.763, val_acc: 41.040, lr: 0.200000
| Epoch [  6/120] Iter[352/352]		Loss: 1.6814 Acc@1: 42.862%, time: 1.175| Epoch: 6/120, val_loss: 1.905, val_acc: 38.860, lr: 0.200000
| Epoch: 6/120, val_loss: 1.633, val_acc: 46.180, lr: 0.200000
| Epoch [  7/120] Iter[352/352]		Loss: 1.2168 Acc@1: 47.084%, time: 1.169| Epoch: 7/120, val_loss: 1.482, val_acc: 43.700, lr: 0.200000
| Epoch: 7/120, val_loss: 1.635, val_acc: 51.960, lr: 0.200000
| Epoch [  8/120] Iter[35

...

| Epoch [ 74/120] Iter[212/352]		Loss: 0.4912 Acc@1: 80.156%, time: 1.869

requests_with_retry encountered retryable exception: 500 Server Error: Internal Server Error for url: https://api.wandb.ai/files/xavierlopeze/Cifar_Experiment/3165tbv4/file_stream. args: ('https://api.wandb.ai/files/xavierlopeze/Cifar_Experiment/3165tbv4/file_stream',), kwargs: {'json': {'files': {'output.log': {'offset': 152, 'content': ['2020-05-12T07:03:08.536417 | Epoch [ 74/120] Iter[190/352]\t\tLoss: 0.4143 Acc@1: 80.144%, time: 1.850\r']}}}}


| Epoch [ 74/120] Iter[218/352]		Loss: 0.5194 Acc@1: 80.100%, time: 1.865

requests_with_retry encountered retryable exception: 500 Server Error: Internal Server Error for url: https://api.wandb.ai/files/xavierlopeze/Cifar_Experiment/3165tbv4/file_stream. args: ('https://api.wandb.ai/files/xavierlopeze/Cifar_Experiment/3165tbv4/file_stream',), kwargs: {'json': {'files': {'output.log': {'offset': 152, 'content': ['2020-05-12T07:03:08.536417 | Epoch [ 74/120] Iter[190/352]\t\tLoss: 0.4143 Acc@1: 80.144%, time: 1.850\r']}}}}


| Epoch [ 74/120] Iter[352/352]		Loss: 0.4643 Acc@1: 79.787%, time: 1.178| Epoch: 74/120, val_loss: 1.518, val_acc: 68.700, lr: 0.200000
| Epoch: 74/120, val_loss: 0.608, val_acc: 90.440, lr: 0.200000
| Epoch [ 75/120] Iter[352/352]		Loss: 0.3654 Acc@1: 79.984%, time: 1.171| Epoch: 75/120, val_loss: 0.875, val_acc: 72.980, lr: 0.200000
| Epoch: 75/120, val_loss: 0.632, val_acc: 90.460, lr: 0.200000
| Epoch [ 76/120] Iter[352/352]		Loss: 0.4473 Acc@1: 80.178%, time: 1.177| Epoch: 76/120, val_loss: 1.234, val_acc: 65.760, lr: 0.200000
| Epoch: 76/120, val_loss: 0.672, val_acc: 90.000, lr: 0.200000
| Epoch [ 77/120] Iter[352/352]		Loss: 0.5459 Acc@1: 80.060%, time: 1.182| Epoch: 77/120, val_loss: 0.843, val_acc: 77.220, lr: 0.200000
| Epoch: 77/120, val_loss: 0.702, val_acc: 90.160, lr: 0.200000
| Epoch [ 78/120] Iter[352/352]		Loss: 0.5129 Acc@1: 79.862%, time: 1.176| Epoch: 78/120, val_loss: 1.137, val_acc: 74.600, lr: 0.200000
| Epoch: 78/120, val_loss: 0.700, val_acc: 90.340, lr: 0.20

In [11]:
print('\nTesting model')
checkpoint = torch.load(config.drive_dir + '/checkpoint/%s.pth.tar' % config.id)
test_net.load_state_dict(checkpoint['state_dict'])
test()



Testing model


OSError: ignored

In [12]:
#This code is useful when the code has crashed without getting to the maximum of epochs (due to google collab session termination)
#essentially we reload weights and following continue the training process

EPOCH = 104 #Input the last epoch computed

print('Load Student')
checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.id + '_student_' + str(EPOCH) + '.pth.tar' )
net.load_state_dict(checkpoint['state_dict'])

print('Load Teacher')

checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.id + '_teacher_' + str(EPOCH) + '.pth.tar' )
tch_net.load_state_dict(checkpoint['state_dict'])

Load Student
Load Teacher


<All keys matched successfully>

In [13]:
init = False
best_acc = 0
for epoch in range(EPOCH, 1 + config.num_epochs):
    train(epoch)
    # Student validation
    std_results = valid(epoch, net)
    record.write(std_results + '\n')
    print(std_results)
    # Teacher validation
    tch_results = valid(epoch, tch_net)
    record.write(tch_results + '\n')
    record.flush()
    print(tch_results)

    save_weights(epoch)

print('\nTesting model')
checkpoint = torch.load(config.drive_dir + '/checkpoint/%s.pth.tar' % config.id)
test_net.load_state_dict(checkpoint['state_dict'])
test()

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2247: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


| Epoch: 104/120, val_loss: 0.666, val_acc: 79.140, lr: 0.020000
| Epoch: 104/120, val_loss: 2.156, val_acc: 83.000, lr: 0.020000
| Epoch [105/120] Iter[352/352]		Loss: 0.2513 Acc@1: 82.360%, time: 1.204| Epoch: 105/120, val_loss: 0.690, val_acc: 80.420, lr: 0.020000
| Epoch: 105/120, val_loss: 1.681, val_acc: 85.020, lr: 0.020000
| Epoch [106/120] Iter[352/352]		Loss: 0.3060 Acc@1: 82.836%, time: 1.207| Epoch: 106/120, val_loss: 0.862, val_acc: 80.140, lr: 0.020000
| Epoch: 106/120, val_loss: 1.427, val_acc: 86.380, lr: 0.020000
| Epoch [107/120] Iter[352/352]		Loss: 0.2364 Acc@1: 83.100%, time: 1.199| Epoch: 107/120, val_loss: 0.858, val_acc: 78.620, lr: 0.020000
| Epoch: 107/120, val_loss: 1.221, val_acc: 87.540, lr: 0.020000
| Epoch [108/120] Iter[352/352]		Loss: 0.2999 Acc@1: 83.402%, time: 1.210| Epoch: 108/120, val_loss: 0.601, val_acc: 81.380, lr: 0.020000
| Epoch: 108/120, val_loss: 1.050, val_acc: 88.060, lr: 0.020000
| Epoch [109/120] Iter[352/352]		Loss: 0.2565 Acc@1: 83.66

In [0]:
# DISCARDING SAMPLES

In [0]:
# RELOAD NETWORKS
EPOCH = 120 #Input the last epoch computed

print('Load Student')
checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.id + '_student_' + str(EPOCH) + '.pth.tar' )
net.load_state_dict(checkpoint['state_dict'])

print('Load Teacher')

checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.id + '_teacher_' + str(EPOCH) + '.pth.tar' )
tch_net.load_state_dict(checkpoint['state_dict'])

print('\nLoad Test')
checkpoint = torch.load(config.drive_dir + '/checkpoint/%s.pth.tar' % config.id)
test_net.load_state_dict(checkpoint['state_dict'])

In [0]:
import pandas
def get_confidence_training(dataloader):
  correct = 0
  total = 0

  target_list = []
  pred        = []
  confidence  = []

  for batch_idx, (inputs, targets) in enumerate(dataloader):
      if use_cuda:
          inputs, targets = inputs.cuda(), targets.cuda()
      with torch.no_grad():
          outputs = test_net(inputs)
          loss = criterion(outputs, targets)  
          _, predicted = torch.max(outputs.data, 1)
          total += targets.size(0)
          correct += predicted.eq(targets.data).cpu().sum()
          
          conf = float(torch.max(outputs.softmax(dim = 1)))
          targ = int(targets.data)
          pd = int(predicted)

          confidence.append(conf)
          target_list.append(targ)
          pred.append(pd)

  # Grab dataframe
  df = pandas.DataFrame()
  df["target_list"] = target_list
  df["pred"] = pred
  df["confidence"] = confidence

  return(df)

In [0]:
#Those parameters are key for discarding
pre_batch = config.batch_size
config.batch_size = 1
config.shuffle = False

# Get the dataloader with no shuffle
loader = dataloader.KeyDataLoader()
train_noshuffle_loader, _, _ = loader.run()


df = get_confidence_training(train_noshuffle_loader)

#get list of directories
train_df = pandas.read_csv(config.data_dir+config.train_dir, header=None)
train_df.columns = ["dir"]

#Merge directories with confidence
DF = df.merge(train_df,how = 'left', left_index = True, right_index = True)

#filter only training samples that have a high enough confidence by the softmax
tau = 0.01
df_filtered = DF[DF.confidence > tau]
df_filtered

#Export the filtered train dataset
df_filtered.to_csv(config.data_dir +'clean_train_key_list_iter2' + config.id + '.txt', header = None, index=False)
config.train_dir = 'clean_train_key_list_iter2' + config.id + '.txt'

#Reset shuffling for next iteration
config.shuffle = True
config.batch_size = pre_batch

# Get the filtered dataset
loader = dataloader.KeyDataLoader()
train_loader, _, _ = loader.run()

#Initialize mentor
mentor_net = get_model()
if use_cuda:
    mentor_net.cuda()

#Get the weights for the mentor
print('\nLoad Mentor')
checkpoint = torch.load(config.drive_dir + '/checkpoint/%s.pth.tar' % config.id)
mentor_net.load_state_dict(checkpoint['state_dict'])

#set mentor parameter on the train funciton
use_mentor = True